In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9857
2,Huelva,Confirmados PDIA 14 días,1530
3,Huelva,Tasa PDIA 14 días,"299,5635769849024"
4,Huelva,Confirmados PDIA 7 días,575
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,122
630,Municipio de Huelva sin especificar,Curados,84


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9857.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2762.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 892 personas en los últimos 7 días 

Un positivo PCR cada 327 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9857.0,575.0,1530.0,510743.0,112.581083,299.563577,104.0
Huelva-Costa,5435.0,363.0,993.0,288115.0,125.991358,344.654044,67.0
Condado-Campiña,3051.0,180.0,426.0,155057.0,116.086342,274.737677,40.0
Huelva (capital),2762.0,161.0,439.0,143663.0,112.067825,305.576244,24.0
Isla Cristina,320.0,46.0,89.0,21264.0,216.328066,418.547780,17.0
Moguer,315.0,35.0,91.0,22088.0,158.457081,411.988410,14.0
Lepe,862.0,45.0,172.0,27431.0,164.047975,627.027815,10.0
Cartaya,319.0,38.0,73.0,19974.0,190.247322,365.475118,8.0
Lucena del Puerto,81.0,20.0,35.0,3371.0,593.295758,1038.267576,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,81.0,20.0,35.0,3371.0,593.295758,1038.267576,4.0
Corteconcepción,18.0,1.0,4.0,548.0,182.481752,729.927007,0.0
Niebla,112.0,11.0,27.0,4117.0,267.184843,655.817343,0.0
Lepe,862.0,45.0,172.0,27431.0,164.047975,627.027815,10.0
Palma del Condado (La),447.0,29.0,67.0,10761.0,269.491683,622.618716,4.0
San Bartolomé de la Torre,40.0,14.0,19.0,3736.0,374.732334,508.565310,4.0
Villalba del Alcor,98.0,8.0,16.0,3338.0,239.664470,479.328939,3.0
Rociana del Condado,113.0,11.0,36.0,7855.0,140.038192,458.306811,2.0
Palos de la Frontera,205.0,22.0,51.0,11289.0,194.879972,451.767207,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,64.0,1.0,12.0,3848.0,25.987526,311.850312,0.0,0.083333
Nerva,86.0,1.0,12.0,5235.0,19.102197,229.226361,0.0,0.083333
Aroche,51.0,2.0,12.0,3073.0,65.082981,390.497885,0.0,0.166667
Bollullos Par del Condado,231.0,3.0,18.0,14272.0,21.020179,126.121076,1.0,0.166667
Cerro de Andévalo (El),36.0,1.0,5.0,2364.0,42.301184,211.505922,0.0,0.200000
Calañas,44.0,1.0,5.0,2791.0,35.829452,179.147259,0.0,0.200000
Corteconcepción,18.0,1.0,4.0,548.0,182.481752,729.927007,0.0,0.250000
Ayamonte,314.0,24.0,93.0,20946.0,114.580349,443.998854,2.0,0.258065
Lepe,862.0,45.0,172.0,27431.0,164.047975,627.027815,10.0,0.261628
